<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Машинное обучение и майнинг данных"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# Занятие 4. Продвинутые методы классификации и регрессии. Переобучение

## Практика. Случайный лес в соревновании Kaggle Inclass по автострахованию. Решение

[Соревнование](https://inclass.kaggle.com/c/hse-addprofeduc-ml-contest), исходное <a href="http://microsoftbi.ru/2015/06/06/hackathon2015ml/">описание</a> задачи. 

Задача бинарной классификации. Имеются автомобили, для которых указан регистрационный номер и марка, и выплаты страховой компании по инцидентам с участием данного автомобиля. Страховая компания для себя решает, много она заплатила или мало. 

Объекты - автомобили.

Признаки:

- Регистрационный номер автомобиля (auto_number, уникальный, строка)
- Марка автомобиля (auto_brand, строка)
- Тип выплаты (too_much) (много/мало, 1 или 0)
- Сумма выплаты при попадании водителя в аварию (compensated, целое положительное число)

### Загрузка и первичный анализ данных

In [27]:
from __future__ import division, print_function
import warnings
warnings.filterwarnings('ignore')
%pylab inline
figsize(12, 8)
import seaborn as sns
import pandas as pd
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import sys
sys.path.append('../../scripts/')
from load_car_insurance_with_region import load_train_and_test

Populating the interactive namespace from numpy and matplotlib


**Считаем обучающую и тестовую выборку, создав объекты Pandas DataFrame.**

In [28]:
train_df, y, test_df = load_train_and_test('../../data/car_insurance_train.csv',
                                          '../../data/car_insurance_test.csv')

In [29]:
train_df.head()

,auto_brand,compensated,region
id,,,
1,2,3200,21
2,5,6500,12
3,2,2100,9
4,2,2000,4
5,2,6100,21


**"Продвинутый" подход для работы с категориальными признаками - это считать статистики (например, max, min, median, квантили и т.д.) количественных признаков по категориальным. Для этого удобно использовать метод groupby pandas.DataFrame (при необходимости посмотрите документацию метода).**

In [33]:
#?train_df.groupby

**Пример. Считаем средние выплаты для каждой марки авто отдельно. Видно, что для разных марок средние выплаты могут существенно отличаться.**

In [30]:
mean_comp_by_brand = {brand_cat[0]: brand_cat[1]['compensated'].mean() 
                      for brand_cat in train_df.groupby("auto_brand")}

In [34]:
mean_comp_by_brand

{0.0: 4200.0,
 1.0: 3366.6666666666665,
 2.0: 7498.480243161094,
 3.0: 14312.5,
 4.0: 9988.235294117647,
 5.0: 7305.882352941177,
 6.0: 10672.222222222223,
 7.0: 5533.333333333333,
 8.0: 15162.5,
 9.0: 9442.857142857143,
 10.0: 5823.809523809524,
 11.0: 3100.0,
 12.0: 8500.0}

**Создайте признаки "Медианные выплаты по маркам", "Средние выплаты по регионам" и т.п. (используйте функцию groupby). Посчитайте статистики mean, median, min и max (можно, конечно, и написать функцию, которая считает заданную статистику для заданного признака). Получится 8 новых признаков. После этого удалите исходные признаки auto_brand и region.**

In [4]:
mean_comp_by_brand = {brand_cat[0]: brand_cat[1]['compensated'].mean() for brand_cat in train_df.groupby("auto_brand")}
median_comp_by_brand = {brand_cat[0]: brand_cat[1]['compensated'].median() for brand_cat in train_df.groupby("auto_brand")}
min_comp_by_brand = {brand_cat[0]: brand_cat[1]['compensated'].min() for brand_cat in train_df.groupby("auto_brand")}
max_comp_by_brand = {brand_cat[0]: brand_cat[1]['compensated'].max() for brand_cat in train_df.groupby("auto_brand")}

mean_comp_by_region = {brand_cat[0]: brand_cat[1]['compensated'].mean() for brand_cat in train_df.groupby("region")}
median_comp_by_region = {brand_cat[0]: brand_cat[1]['compensated'].median() for brand_cat in train_df.groupby("region")}
min_comp_by_region = {brand_cat[0]: brand_cat[1]['compensated'].min() for brand_cat in train_df.groupby("region")}
max_comp_by_region = {brand_cat[0]: brand_cat[1]['compensated'].max() for brand_cat in train_df.groupby("region")}

In [5]:
train_df['mean_by_brand'] = [mean_comp_by_brand[i] for i in train_df['auto_brand']]
test_df['mean_by_brand'] = [mean_comp_by_brand[i] for i in test_df['auto_brand']]
train_df['median_by_brand'] = [median_comp_by_brand[i] for i in train_df['auto_brand']]
test_df['median_by_brand'] = [median_comp_by_brand[i] for i in test_df['auto_brand']]
train_df['min_by_brand'] = [min_comp_by_brand[i] for i in train_df['auto_brand']]
test_df['min_by_brand'] = [min_comp_by_brand[i] for i in test_df['auto_brand']]
train_df['max_by_brand'] = [max_comp_by_brand[i] for i in train_df['auto_brand']]
test_df['max_by_brand'] = [max_comp_by_brand[i] for i in test_df['auto_brand']]

train_df['mean_by_region'] = [mean_comp_by_region[i] for i in train_df['region']]
test_df['mean_by_region'] = [mean_comp_by_region[i] for i in test_df['region']]
train_df['median_by_region'] = [median_comp_by_region[i] for i in train_df['region']]
test_df['median_by_region'] = [median_comp_by_region[i] for i in test_df['region']]
train_df['min_by_region'] = [min_comp_by_region[i] for i in train_df['region']]
test_df['min_by_region'] = [min_comp_by_region[i] for i in test_df['region']]
train_df['max_by_region'] = [max_comp_by_region[i] for i in train_df['region']]
test_df['max_by_region'] = [max_comp_by_region[i] for i in test_df['region']]

In [6]:
train_df.head()

,auto_brand,compensated,region,mean_by_brand,median_by_brand,min_by_brand,max_by_brand,mean_by_region,median_by_region,min_by_region,max_by_region
id,,,,,,,,,,,
1,2,3200,21,7498.480243,2500,100,180000,8638.888889,3100,100,103600
2,5,6500,12,7305.882353,6500,500,22400,5608.333333,5000,1100,13000
3,2,2100,9,7498.480243,2500,100,180000,5650.000000,4700,500,18800
4,2,2000,4,7498.480243,2500,100,180000,2587.692308,1500,100,46200
5,2,6100,21,7498.480243,2500,100,180000,8638.888889,3100,100,103600


In [7]:
train_df.drop(['region', 'auto_brand'], axis=1, inplace=True)

In [8]:
test_df.drop(['region', 'auto_brand'], axis=1, inplace=True)

In [9]:
train_df.head()

,compensated,mean_by_brand,median_by_brand,min_by_brand,max_by_brand,mean_by_region,median_by_region,min_by_region,max_by_region
id,,,,,,,,,
1,3200,7498.480243,2500,100,180000,8638.888889,3100,100,103600
2,6500,7305.882353,6500,500,22400,5608.333333,5000,1100,13000
3,2100,7498.480243,2500,100,180000,5650.000000,4700,500,18800
4,2000,7498.480243,2500,100,180000,2587.692308,1500,100,46200
5,6100,7498.480243,2500,100,180000,8638.888889,3100,100,103600


In [10]:
test_df.head()

,compensated,mean_by_brand,median_by_brand,min_by_brand,max_by_brand,mean_by_region,median_by_region,min_by_region,max_by_region
id,,,,,,,,,
1,6000,7498.480243,2500,100,180000,8638.888889,3100,100,103600
2,3000,9988.235294,2000,300,145000,19044.444444,21000,1000,49200
3,5000,7498.480243,2500,100,180000,8806.451613,8000,500,32000
4,4600,7498.480243,2500,100,180000,15056.410256,7600,100,180000
5,3000,9988.235294,2000,300,145000,1650.000000,1650,300,3000


**Определите с помощью случайного леса важность имеющихся признаков. Здесь можно брать много (например, 1000) глубоких (например, глубины 5) деревьев решений. Используйте параметр random_state=42.**

In [25]:
forest = RandomForestClassifier(n_estimators=1000,
                               max_depth=5,
                               random_state=42)
forest.fit(train_df, y)
pd.DataFrame(forest.feature_importances_,
             index=train_df.columns,
            columns=['Importance']).sort(['Importance'], ascending=False)

,Importance
compensated,0.654327
median_by_region,0.084812
mean_by_region,0.084311
max_by_region,0.055484
max_by_brand,0.027447
median_by_brand,0.026239
mean_by_brand,0.025839
min_by_region,0.021073
min_by_brand,0.020468


**Обучите на имеющейся выборке с 3 "лучшими" признаками случайный лес, переберите параметры глубины дерева от 1 до 4, а число деревьев - 100 или 500. Используйте 5-кратную кросс-валидацию и параметр random_state=42.**

In [12]:
# tree params for grid search
tree_params = {'n_estimators': [100, 500],
               'max_depth': list(range(1,4)),}
               #'min_samples_leaf': list(range(1,5))}

locally_best_clf = GridSearchCV(RandomForestClassifier(random_state=42), 
                                 tree_params, 
                                 verbose=True, n_jobs=1, cv=5)
locally_best_clf.fit(train_df[['compensated', 'median_by_region', 'mean_by_region']], y)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    6.2s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_estimators': [100, 500], 'max_depth': [1, 2, 3]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=True)

In [13]:
locally_best_clf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [14]:
locally_best_clf.best_params_

{'max_depth': 2, 'n_estimators': 100}

In [15]:
locally_best_clf.best_score_

0.70991432068543447

**Обучите на всей выборке случайный лес, с лучшими параметрами, определенными ранее. Используйте также параметр random_state=42.**

In [16]:
final_model = RandomForestClassifier(n_estimators=100,
                                          max_depth=2,
                                     random_state=42)
final_model.fit(train_df[['compensated', 
                          'median_by_region', 'mean_by_region']], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

### Предсказание выплат для тестовой выборки

**Сделайте прогноз для объектов тестовой выборки.**

In [17]:
predicted_labels = final_model.predict(test_df[['compensated', 
                                                'median_by_region', 
                                                'mean_by_region']])

**Сравним сразу с ответами (в качестве демонстрации, в реальной задаче ответы, конечно, неизвестны).**

In [18]:
from sklearn.metrics import roc_auc_score

try:
    expected_labels_df = pd.read_csv("../../data/car_insurance_test_labels.csv",
                                     header=0, index_col=0)
    expected_labels = expected_labels_df['too_much']
    print(roc_auc_score(predicted_labels, expected_labels))
except OSError:
    print("You shouldn't know the answers, but this results in 0.752 ROC AUC")

0.751937984496


**Запишите ответы в csv-файл и отправьте решение на Kaggle.**

In [19]:
# turn predictions into data frame and save as csv file
predicted_df = pd.DataFrame(predicted_labels,
                            index = np.arange(1, test_df.shape[0] + 1),
                            columns=["too_much"])
predicted_df.to_csv("../../output/rf_with_region_n100_depth2.csv", index_label="id")

**для Weka**

In [20]:
y_nominal = pd.Series(['OK' if label == 0 else 'too_much' 
                       for label in y], name='target', 
                      index=range(1, y.shape[0]+1))
test_labels = pd.read_csv("../../data/car_insurance_test_labels.csv",
        header=0, index_col=0)
y_test_nominal = pd.Series(['OK' if label == 0 else 'too_much' 
                            for label in np.array(test_labels)], name='target', 
                      index=range(1, test_labels.shape[0]+1))
pd.concat([train_df, y_nominal], 
          axis=1).to_csv('../../output/car_insur_train_modified.csv')
pd.concat([test_df, y_test_nominal], 
          axis=1).to_csv('../../output/car_insur_test_modified.csv')

## Ссылки:
- <a href="https://inclass.kaggle.com/c/hse-addprofeduc-ml-contest">Соревнование</a> на сайте Kaggle Inclass
- Исходное <a href="http://microsoftbi.ru/2015/06/06/hackathon2015ml/">описание</a> задачи
- [Документация](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) Scikit-learn по классу RandomForestClassifier